# Problem 2

I originally wanted to do a diffusion model for this problem, but I am running out of time and I am not condfident that I will get it to work by the deadline. So, I will reuse the CVAE architecture I used in problem 1 for autoregressive purposes.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


kernel_size = 4 # (4, 4) kernel
init_channels = 8 # initial number of filters
image_channels = 2
latent_dim = 16 # latent dimension for sampling

This is a modified CVAE architecture from problem 1 to allow for concatenating the previous time step with the latent space.

In [4]:
class ConvVAE(nn.Module):
    def __init__(self, image_channels=2, init_channels=32, kernel_size=3, latent_dim=64, additional_data_dim=2*128*256):
        super(ConvVAE, self).__init__()

        # Encoder
        self.enc1 = nn.Conv2d(image_channels, init_channels, kernel_size=kernel_size, stride=2, padding=1)  # Output: [batch, 32, 64, 128]
        self.enc2 = nn.Conv2d(init_channels, init_channels * 2, kernel_size=kernel_size, stride=2, padding=1)  # Output: [batch, 64, 32, 64]
        self.enc3 = nn.Conv2d(init_channels * 2, init_channels * 4, kernel_size=kernel_size, stride=2, padding=1)  # Output: [batch, 128, 16, 32]
        self.enc4 = nn.Conv2d(init_channels * 4, 256, kernel_size=kernel_size, stride=2, padding=1)  # Output: [batch, 256, 8, 16]

        # Latent space
        self.fc_mu = nn.Linear(256 * 8 * 16, latent_dim)
        self.fc_log_var = nn.Linear(256 * 8 * 16, latent_dim)
        self.fc2 = nn.Linear(latent_dim + additional_data_dim, 256 * 8 * 16)  # Adjusted for concatenated input size

        # Decoder
        self.dec1 = nn.ConvTranspose2d(256, init_channels * 4, kernel_size=kernel_size, stride=2, padding=1, output_padding=1)  # Output: [batch, 128, 16, 32]
        self.dec2 = nn.ConvTranspose2d(init_channels * 4, init_channels * 2, kernel_size=kernel_size, stride=2, padding=1, output_padding=1)  # Output: [batch, 64, 32, 64]
        self.dec3 = nn.ConvTranspose2d(init_channels * 2, init_channels, kernel_size=kernel_size, stride=2, padding=1, output_padding=1)  # Output: [batch, 32, 64, 128]
        self.dec4 = nn.ConvTranspose2d(init_channels, image_channels, kernel_size=kernel_size, stride=2, padding=1, output_padding=1)  # Output: [batch, 2, 128, 256]

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, additional_data):
        # Encoding
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))

        # Flatten and pass through fully connected layers
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)

        # Reparameterization trick
        z = self.reparameterize(mu, log_var)

        # Flatten the additional data (2, 128, 256) to (batch_size, 2 * 128 * 256)
        additional_data_flat = additional_data.view(batch_size, -1)

        # Concatenate the latent vector `z` with the additional data
        z = torch.cat((z, additional_data_flat), dim=1)

        # Pass the concatenated vector through the fully connected layer
        z = F.relu(self.fc2(z))  # Reduce the combined vector size to [batch_size, 256 * 8 * 16]
        z = z.view(-1, 256, 8, 16)  # Reshape to [batch_size, 256, 8, 16]

        # Decoder
        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        reconstruction = torch.sigmoid(self.dec4(x))

        return reconstruction, mu, log_var



# Inference: After training, the model can be used like this:
def predict_next_time_step(model, previous_time_step_data):
    """
    Function to predict the next time step given the previous time step data (geopotential).

    model: The trained ConvVAE model.
    previous_time_step_data: Geopotential at the previous time step (shape: batch_size, 2, 91, 180).

    Returns: Reconstructed geopotential for the next time step.
    """
    model.eval()  # Set the model to evaluation mode

    # Ensure the input data is a tensor
    previous_time_step_data = torch.tensor(previous_time_step_data).float()

    # No need for labels, just pass the previous time step data as additional data
    with torch.no_grad():
        reconstructed_data, _, _ = model(previous_time_step_data, previous_time_step_data)

    return reconstructed_data


I think this will work if the additional data from the previous time step is flattened.

In [5]:
import xarray as xr

data_root = "/content/drive/MyDrive/Colab Notebooks/data_AM160_final/"

data_1979 = xr.open_dataset(data_root + "z1979.nc")
data_1980 = xr.open_dataset(data_root + "z1980.nc")
data_1981 = xr.open_dataset(data_root + "z1981.nc")
data_1983 = xr.open_dataset(data_root + "z1983.nc")
data_list = [data_1979, data_1980, data_1981, data_1983]
data_list_np = [dataset["z"].values for dataset in data_list]

In [6]:
import numpy as np

X = np.concatenate(data_list_np, axis=0)

X_min = np.min(X)
X_max = np.max(X)

X_norm = (X - X_min) / (X_max - X_min)

In [7]:
X_tensor = torch.tensor(X_norm, dtype=torch.float32)

del X, X_norm

In [8]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [9]:
# Function to calculate the next power of 2 greater than or equal to the size
def next_power_of_2(x):
    return 2 ** np.ceil(np.log2(x)).astype(int)

# Function to pad an image to the next power of 2 in both dimensions
def pad_to_power_of_2(x):
    batch_size, channels, height, width = x.size()

    # Calculate the next power of 2 for height and width
    target_height = next_power_of_2(height)
    target_width = next_power_of_2(width)

    # Calculate padding needed
    pad_height = target_height - height
    pad_width = target_width - width

    # Padding should be applied symmetrically
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left

    # Apply padding
    padded_x = F.pad(x, (pad_left, pad_right, pad_top, pad_bottom), mode='constant', value=0)

    return padded_x


# Pad the input tensor to the next power of 2
X_padded = pad_to_power_of_2(X_tensor)
print(f"Padded input size: {X_padded.size()}")

Padded input size: torch.Size([5844, 2, 128, 256])


In [10]:
X_minus = torch.roll(X_padded, shifts=-1, dims=0)
X_minus.size()

torch.Size([5844, 2, 128, 256])

In [11]:
torch.sum(X_padded[1] == X_minus[0])

tensor(65536)

In [ ]:
import torch.optim as optim
# Initialize the network, loss function, and optimizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ConvVAE().to(device)
print("check")
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 100
trainN = 5844
# Train the network
num_epochs = 50
LOSS = []

for epoch in range(num_epochs):
    for iter in range (0, trainN, batch_size):
        if (iter % 1000 == 0):
         print('iteration number,',iter)
        batch_X = X_padded[iter:iter+batch_size]
        batch_X_minus = X_minus[iter:iter+batch_size]
        #print(batch_X.size())
        optimizer.zero_grad()
        outputs = model(batch_X.to(device).float(), batch_X_minus.to(device).float())
        reconstruction, mu, log_var = outputs
        #print(reconstruction.size())
        bce_loss = criterion(reconstruction, batch_X.to(device).float())
        loss = final_loss(bce_loss, mu, log_var)
        loss.backward()
        optimizer.step()
    LOSS.append(loss.detach().cpu().numpy())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


g

g

g

g

g

g

g

g

g

g

g

g

g

g

g

g

g

g